<span>
<img src="http://ash.readthedocs.io/en/latest/_static/ash.png" width="260px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="https://andreafailla.github.io">Andrea Failla</a><br/>
<b>Python version:</b>  3.9<br/>
<b>ASH version:</b>  1.0.0<br/>
<b>Last update:</b> November 2025
</span>

<a id="multi-ego-networks"></a>
# Multi-Ego Networks

In [6]:
import sys
sys.path.insert(0, '../')

<a id="table-of-contents"></a>
# Table of Contents

  - [Introduction](#introduction)
  - [Basic Concepts](#basic-concepts)
    - [What is an Ego Network?](#what-is-an-ego-network)
    - [What is a Multi-Ego Network?](#what-is-a-multi-ego-network)
  - [Extracting Multi-Ego Networks](#extracting-multi-ego-networks)
    - [Standard Multi-Ego Network](#standard-multi-ego-network)
    - [Fractured Multi-Ego Network](#fractured-multi-ego-network)
    - [Core Multi-Ego Network](#core-multi-ego-network)
  - [Temporal Multi-Ego Networks](#temporal-multi-ego-networks)
  - [Comparing Multi-Ego Networks](#comparing-multi-ego-networks)
    - [Jaccard Similarity](#jaccard-similarity)
    - [Minimum Overlapping Similarity](#minimum-overlapping-similarity)
    - [Delta Similarity](#delta-similarity)
  - [Practical Examples](#practical-examples)

<a id="introduction"></a>
## Introduction

Multi-Ego Networks extend the classical concept of ego networks from simple graphs to temporal hypergraphs. While a traditional ego network focuses on a single node and its immediate neighbors, Multi-Ego Networks allow us to study the local structure around **multiple focal nodes** simultaneously, providing a richer understanding of group-centric patterns in higher-order interactions.

This tutorial will guide you through the extraction, analysis, and comparison of Multi-Ego Networks in ASH.

[🔝 To top](#table-of-contents)

<a id="basic-concepts"></a>
## Basic Concepts

<a id="what-is-an-ego-network"></a>
### What is an Ego Network?

An **ego network** is a subnetwork centered around a single focal node (the "ego"). In a simple graph, it consists of:
- The ego node itself
- All nodes directly connected to the ego (called "alters")
- All edges between these nodes

In a hypergraph, the ego network consists of all hyperedges that contain the ego node. This is accessible in ASH via the `star()` method:

```python
h = ASH()
# ... add hyperedges ...
ego_edges = h.star(node_id=1, start=0, end=5)
```

[🔝 To top](#table-of-contents)

<a id="what-is-a-multi-ego-network"></a>
### What is a Multi-Ego Network?

A **Multi-Ego Network** generalizes the ego network concept to **multiple focal nodes** (egos). Instead of centering the analysis on a single node, we consider a set of nodes U = {u₁, u₂, ..., uₖ} and extract the local structure around all of them.

The `multiego` module in ASH provides three variants:

1. **Standard Multi-Ego**: All hyperedges containing **at least one** node from U
2. **Fractured Multi-Ego**: All hyperedges containing **at least α·|U|** nodes from U
3. **Core Multi-Ego**: All hyperedges where nodes from U represent **at least β fraction** of the hyperedge size

These variants allow fine-grained control over what constitutes "local structure" for a group of nodes.

[🔝 To top](#table-of-contents)

<a id="extracting-multi-ego-networks"></a>
## Extracting Multi-Ego Networks

Let's create a sample temporal hypergraph and explore the different types of Multi-Ego Networks.

In [7]:
from ash_model import ASH
from ash_model.multiego import (
    get_multiego,
    get_fractured_multiego,
    get_core_multiego
)

# Create a temporal hypergraph
h = ASH()

# Add hyperedges at different time points
# Time 0
h.add_hyperedge([1, 2, 3], start=0, end=0)
h.add_hyperedge([2, 3, 4], start=0, end=0)
h.add_hyperedge([1, 4, 5], start=0, end=0)

# Time 1
h.add_hyperedge([1, 2, 5], start=1, end=1)
h.add_hyperedge([3, 4, 5, 6], start=1, end=1)
h.add_hyperedge([1, 3], start=1, end=1)

# Time 2
h.add_hyperedge([2, 4, 6],  start=2, end=2)
h.add_hyperedge([1, 2, 3, 4], start=2, end=2)

print(f"Hypergraph created with {h.number_of_nodes()} nodes and {h.number_of_hyperedges()} hyperedges")
print(f"Time range: {h.temporal_snapshots_ids()}")

Hypergraph created with 6 nodes and 8 hyperedges
Time range: [0, 1, 2]


<a id="standard-multi-ego-network"></a>
### Standard Multi-Ego Network

The standard Multi-Ego Network includes all hyperedges that contain **at least one** node from the ego set U.

**Use case**: Finding all interactions where at least one member of a team/group participates.

In [8]:
# Define our ego set
U = {1, 2, 3}

# Extract Multi-Ego Network at time 0
multiego_t0 = get_multiego(h, U, start=0)
print(f"Multi-Ego Network for U={U} at time 0:")
for i, edge in enumerate(multiego_t0, 1):
    print(f"  {i}. {edge}")

# Extract Multi-Ego Network for time window [0, 1]
multiego_window = get_multiego(h, U, start=0, end=1)
print(f"\nMulti-Ego Network for U={U} in window [0,1]:")
for i, edge in enumerate(multiego_window, 1):
    print(f"  {i}. {edge}")

print(f"\nTotal hyperedges in Multi-Ego: {len(multiego_window)}")

Multi-Ego Network for U={1, 2, 3} at time 0:
  1. {1, 2, 3}
  2. {1, 4, 5}
  3. {2, 3, 4}

Multi-Ego Network for U={1, 2, 3} in window [0,1]:
  1. {1, 2, 3}
  2. {1, 3}
  3. {1, 4, 5}
  4. {2, 3, 4}
  5. {3, 4, 5, 6}
  6. {1, 2, 5}

Total hyperedges in Multi-Ego: 6


<a id="fractured-multi-ego-network"></a>
### Fractured Multi-Ego Network

The Fractured Multi-Ego Network includes hyperedges where **at least α·|U|** nodes from U are present. The parameter α ∈ (0, 1] controls the "strength" of the ego presence.

**Use case**: Finding interactions where a significant fraction of a team is present (e.g., meetings with at least 50% of core team members).

In [9]:
# Extract Fractured Multi-Ego with different alpha values
U = {1, 2, 3}

# alpha = 0.33: at least 1 node from U (33% of 3)
fractured_033 = get_fractured_multiego(h, U, start=0, end=1, alpha=0.33)
print(f"Fractured Multi-Ego (α=0.33) for U={U}:")
for edge in fractured_033:
    print(f"  {edge}")

# alpha = 0.67: at least 2 nodes from U (67% of 3)
fractured_067 = get_fractured_multiego(h, U, start=0, end=1, alpha=0.67)
print(f"\nFractured Multi-Ego (α=0.67) for U={U}:")
for edge in fractured_067:
    print(f"  {edge}")

# alpha = 1.0: all nodes from U must be present
fractured_100 = get_fractured_multiego(h, U, start=0, end=1, alpha=1.0)
print(f"\nFractured Multi-Ego (α=1.0) for U={U}:")
for edge in fractured_100:
    print(f"  {edge}")

print(f"\nSummary: α=0.33 → {len(fractured_033)} edges, α=0.67 → {len(fractured_067)} edges, α=1.0 → {len(fractured_100)} edges")

Fractured Multi-Ego (α=0.33) for U={1, 2, 3}:
  {1, 2, 3}
  {1, 3}
  {1, 4, 5}
  {2, 3, 4}
  {3, 4, 5, 6}
  {1, 2, 5}

Fractured Multi-Ego (α=0.67) for U={1, 2, 3}:
  {1, 2, 3}

Fractured Multi-Ego (α=1.0) for U={1, 2, 3}:
  {1, 2, 3}

Summary: α=0.33 → 6 edges, α=0.67 → 1 edges, α=1.0 → 1 edges


<a id="core-multi-ego-network"></a>
### Core Multi-Ego Network

The Core Multi-Ego Network includes hyperedges where nodes from U represent **at least β fraction** of the hyperedge size. The parameter β ∈ (0, 1] controls how "dominated" the hyperedge must be by the ego set.

**Use case**: Finding small interactions where the ego group dominates (e.g., committees where core members are the majority).

In [10]:
# Extract Core Multi-Ego with different beta values
U = {1, 2, 3}

# beta = 0.5: U nodes are at least 50% of the hyperedge
core_050 = get_core_multiego(h, U, start=0, end=1, beta=0.5)
print(f"Core Multi-Ego (β=0.5) for U={U}:")
for edge in core_050:
    u_in_edge = len(set(edge).intersection(U))
    print(f"  {edge} (U contributes {u_in_edge}/{len(edge)} nodes = {u_in_edge/len(edge):.1%})")

# beta = 0.75: U nodes are at least 75% of the hyperedge
core_075 = get_core_multiego(h, U, start=0, end=1, beta=0.75)
print(f"\nCore Multi-Ego (β=0.75) for U={U}:")
for edge in core_075:
    u_in_edge = len(set(edge).intersection(U))
    print(f"  {edge} (U contributes {u_in_edge}/{len(edge)} nodes = {u_in_edge/len(edge):.1%})")

print(f"\nSummary: β=0.5 → {len(core_050)} edges, β=0.75 → {len(core_075)} edges")

Core Multi-Ego (β=0.5) for U={1, 2, 3}:
  {1, 2, 3} (U contributes 3/3 nodes = 100.0%)
  {1, 3} (U contributes 2/2 nodes = 100.0%)
  {2, 3, 4} (U contributes 2/3 nodes = 66.7%)
  {1, 2, 5} (U contributes 2/3 nodes = 66.7%)

Core Multi-Ego (β=0.75) for U={1, 2, 3}:
  {1, 2, 3} (U contributes 3/3 nodes = 100.0%)
  {1, 3} (U contributes 2/2 nodes = 100.0%)

Summary: β=0.5 → 4 edges, β=0.75 → 2 edges


<a id="temporal-multi-ego-networks"></a>
## Temporal Multi-Ego Networks

One of the key strengths of Multi-Ego Networks in ASH is the ability to study how the local structure around a group evolves over time.

In [11]:
# Track Multi-Ego evolution over time
U = {1, 2, 3}

print(f"Multi-Ego Network evolution for U={U}:\n")

for t in h.temporal_snapshots_ids():
    multiego_t = get_multiego(h, U, start=t)
    print(f"Time {t}: {len(multiego_t)} hyperedges")
    for edge in multiego_t:
        print(f"  {edge}")
    print()

# Compare snapshots
multiego_early = get_multiego(h, U, start=0)
multiego_late = get_multiego(h, U, start=2)

print(f"Early snapshot (t=0): {len(multiego_early)} hyperedges")
print(f"Late snapshot (t=2): {len(multiego_late)} hyperedges")

Multi-Ego Network evolution for U={1, 2, 3}:

Time 0: 3 hyperedges
  {1, 2, 3}
  {1, 4, 5}
  {2, 3, 4}

Time 1: 3 hyperedges
  {1, 3}
  {3, 4, 5, 6}
  {1, 2, 5}

Time 2: 2 hyperedges
  {1, 2, 3, 4}
  {2, 4, 6}

Early snapshot (t=0): 3 hyperedges
Late snapshot (t=2): 2 hyperedges


<a id="comparing-multi-ego-networks"></a>
## Comparing Multi-Ego Networks

The `multiego` module provides three similarity measures to compare Multi-Ego Networks across time, different ego sets, or different conditions.

In [12]:
from ash_model.multiego import (
    jaccard_similarity,
    minimum_overlapping_similarity,
    delta_similarity
)

# Get Multi-Ego Networks at different times
U = {1, 2, 3}
multiego_t0 = get_multiego(h, U, start=0)
multiego_t1 = get_multiego(h, U, start=1)
multiego_t2 = get_multiego(h, U, start=2)

print(f"Comparing Multi-Ego Networks for U={U} across time:\n")
print(f"Time 0: {multiego_t0}")
print(f"Time 1: {multiego_t1}")
print(f"Time 2: {multiego_t2}")

Comparing Multi-Ego Networks for U={1, 2, 3} across time:

Time 0: [{1, 2, 3}, {1, 4, 5}, {2, 3, 4}]
Time 1: [{1, 3}, {3, 4, 5, 6}, {1, 2, 5}]
Time 2: [{1, 2, 3, 4}, {2, 4, 6}]


<a id="jaccard-similarity"></a>
### Jaccard Similarity

Jaccard similarity measures the overlap as: **|intersection| / |union|**

This is a strict measure - hyperedges must match exactly (same set of nodes).

In [13]:
# Compute Jaccard similarity between time snapshots
jac_01 = jaccard_similarity(multiego_t0, multiego_t1)
jac_12 = jaccard_similarity(multiego_t1, multiego_t2)
jac_02 = jaccard_similarity(multiego_t0, multiego_t2)

print(f"Jaccard similarity:")
print(f"  t0 vs t1: {jac_01:.3f}")
print(f"  t1 vs t2: {jac_12:.3f}")
print(f"  t0 vs t2: {jac_02:.3f}")

# Interpretation
if jac_01 > 0.5:
    print("\n→ High overlap between consecutive snapshots")
else:
    print("\n→ Low overlap - the Multi-Ego structure changed significantly")

Jaccard similarity:
  t0 vs t1: 0.000
  t1 vs t2: 0.000
  t0 vs t2: 0.000

→ Low overlap - the Multi-Ego structure changed significantly


<a id="minimum-overlapping-similarity"></a>
### Minimum Overlapping Similarity

Minimum overlapping similarity measures: **|intersection| / min(|set1|, |set2|)**

This is more lenient than Jaccard - it normalizes by the smaller set.

In [14]:
# Compute minimum overlapping similarity
overlap_01 = minimum_overlapping_similarity(multiego_t0, multiego_t1)
overlap_12 = minimum_overlapping_similarity(multiego_t1, multiego_t2)
overlap_02 = minimum_overlapping_similarity(multiego_t0, multiego_t2)

print(f"Minimum overlapping similarity:")
print(f"  t0 vs t1: {overlap_01:.3f}")
print(f"  t1 vs t2: {overlap_12:.3f}")
print(f"  t0 vs t2: {overlap_02:.3f}")

print(f"\nComparison with Jaccard:")
print(f"  Jaccard(t0,t1)={jac_01:.3f} vs Overlap(t0,t1)={overlap_01:.3f}")
print(f"  → Overlap is always ≥ Jaccard (more lenient)")

Minimum overlapping similarity:
  t0 vs t1: 0.000
  t1 vs t2: 0.000
  t0 vs t2: 0.000

Comparison with Jaccard:
  Jaccard(t0,t1)=0.000 vs Overlap(t0,t1)=0.000
  → Overlap is always ≥ Jaccard (more lenient)


<a id="delta-similarity"></a>
### Delta Similarity

Delta similarity is a **soft matching** measure that considers node-level overlap between hyperedges. It finds the best pairing between hyperedges based on their node similarity.

This is useful when hyperedges don't match exactly but share many nodes.

In [15]:
# Compute delta similarity
delta_01 = delta_similarity(multiego_t0, multiego_t1)
delta_12 = delta_similarity(multiego_t1, multiego_t2)
delta_02 = delta_similarity(multiego_t0, multiego_t2)

print(f"Delta similarity (soft matching):")
print(f"  t0 vs t1: {delta_01:.3f}")
print(f"  t1 vs t2: {delta_12:.3f}")
print(f"  t0 vs t2: {delta_02:.3f}")

print(f"\nComparison of all three measures (t0 vs t1):")
print(f"  Jaccard: {jac_01:.3f} (strict)")
print(f"  Min Overlap: {overlap_01:.3f} (lenient)")
print(f"  Delta: {delta_01:.3f} (soft matching)")

Delta similarity (soft matching):
  t0 vs t1: 0.522
  t1 vs t2: 0.300
  t0 vs t2: 0.417

Comparison of all three measures (t0 vs t1):
  Jaccard: 0.000 (strict)
  Min Overlap: 0.000 (lenient)
  Delta: 0.522 (soft matching)


<a id="practical-examples"></a>
## Practical Examples

Let's explore some practical scenarios where Multi-Ego Networks are useful.

### Example 1: Comparing Different Ego Sets

How does the local structure differ for different groups of nodes?

In [16]:
# Define two different ego sets
U1 = {1, 2, 3}  # First group
U2 = {4, 5, 6}  # Second group

# Get their Multi-Ego Networks
multiego_U1 = get_multiego(h, U1, start=0, end=2)
multiego_U2 = get_multiego(h, U2, start=0, end=2)

print(f"Multi-Ego for U1={U1}: {len(multiego_U1)} hyperedges")
print(f"Multi-Ego for U2={U2}: {len(multiego_U2)} hyperedges")

# Compare the two groups
sim = jaccard_similarity(multiego_U1, multiego_U2)
print(f"\nSimilarity between the two groups: {sim:.3f}")

if sim < 0.2:
    print("→ The two groups operate in largely separate regions of the hypergraph")
else:
    print("→ The two groups have overlapping neighborhoods")

Multi-Ego for U1={1, 2, 3}: 8 hyperedges
Multi-Ego for U2={4, 5, 6}: 6 hyperedges

Similarity between the two groups: 0.750
→ The two groups have overlapping neighborhoods


### Example 2: Tracking Group Cohesion Over Time

How cohesive is a group? Do they participate in many hyperedges together?

In [17]:
# Define a group and track its cohesion
U = {1, 2, 3}

print(f"Group cohesion analysis for U={U}:\n")

for t in h.temporal_snapshots_ids():
    # Standard: any member participates
    standard = get_multiego(h, U, start=t)
    
    # Fractured: at least 2 members (67%)
    fractured = get_fractured_multiego(h, U, start=t, alpha=0.67)
    
    # Core: members dominate (>50%)
    core = get_core_multiego(h, U, start=t, beta=0.5)
    
    cohesion_ratio = len(fractured) / len(standard) if len(standard) > 0 else 0
    
    print(f"Time {t}:")
    print(f"  Any member: {len(standard)} hyperedges")
    print(f"  Multiple members (≥67%): {len(fractured)} hyperedges")
    print(f"  Dominant (≥50%): {len(core)} hyperedges")
    print(f"  Cohesion ratio: {cohesion_ratio:.2f}")
    print()

print("Interpretation:")
print("  High cohesion ratio → Group members often appear together")
print("  Low cohesion ratio → Group members participate independently")

Group cohesion analysis for U={1, 2, 3}:

Time 0:
  Any member: 3 hyperedges
  Multiple members (≥67%): 1 hyperedges
  Dominant (≥50%): 2 hyperedges
  Cohesion ratio: 0.33

Time 1:
  Any member: 3 hyperedges
  Multiple members (≥67%): 0 hyperedges
  Dominant (≥50%): 2 hyperedges
  Cohesion ratio: 0.00

Time 2:
  Any member: 2 hyperedges
  Multiple members (≥67%): 1 hyperedges
  Dominant (≥50%): 1 hyperedges
  Cohesion ratio: 0.50

Interpretation:
  High cohesion ratio → Group members often appear together
  Low cohesion ratio → Group members participate independently


### Example 3: Identifying Group Influence Zones

Where does a group have the most influence? Find hyperedges dominated by the group.

In [18]:
# Find where the group dominates
U = {1, 2, 3}

# Get Core Multi-Ego with high beta
dominated = get_core_multiego(h, U, start=0, end=2, beta=0.66)

print(f"Hyperedges dominated by U={U} (β≥0.66):\n")

for edge in dominated:
    u_nodes = set(edge).intersection(U)
    other_nodes = set(edge) - U
    
    print(f"  {edge}")
    print(f"    → From U: {u_nodes}")
    print(f"    → Others: {other_nodes if other_nodes else 'none'}")
    print(f"    → U contribution: {len(u_nodes)}/{len(edge)} = {len(u_nodes)/len(edge):.1%}")
    print()

print(f"Total influence zones: {len(dominated)} hyperedges")

Hyperedges dominated by U={1, 2, 3} (β≥0.66):

  {1, 2, 3}
    → From U: {1, 2, 3}
    → Others: none
    → U contribution: 3/3 = 100.0%

  {1, 3}
    → From U: {1, 3}
    → Others: none
    → U contribution: 2/2 = 100.0%

  {1, 2, 3, 4}
    → From U: {1, 2, 3}
    → Others: {4}
    → U contribution: 3/4 = 75.0%

  {2, 3, 4}
    → From U: {2, 3}
    → Others: {4}
    → U contribution: 2/3 = 66.7%

  {1, 2, 5}
    → From U: {1, 2}
    → Others: {5}
    → U contribution: 2/3 = 66.7%

Total influence zones: 5 hyperedges


## Summary

Multi-Ego Networks provide powerful tools for analyzing group-centric structures in temporal hypergraphs:

**Key Concepts:**
- **Standard Multi-Ego**: Captures all interactions involving any group member
- **Fractured Multi-Ego**: Requires minimum group presence (α·|U| members)
- **Core Multi-Ego**: Requires group dominance (β fraction of hyperedge)

**Similarity Measures:**
- **Jaccard**: Strict matching (exact hyperedge equality)
- **Minimum Overlapping**: Lenient matching (normalized by smaller set)
- **Delta**: Soft matching (node-level similarity)

**Applications:**
- Tracking group evolution over time
- Comparing different groups
- Measuring group cohesion
- Identifying influence zones
- Studying collaborative patterns

**Next Steps:**
- Explore the [walks tutorial](02-walks.ipynb) for path-based analysis
- Check the [attribute analysis tutorial](01-attribute_analysis.ipynb) for combining Multi-Ego with node attributes
- See the [generators tutorial](03-generators.ipynb) for creating synthetic hypergraphs

[🔝 To top](#table-of-contents)